In [2]:
DBHOST="127.0.0.1"
DBPORT="6432"
DBUSER="fraud"
DBPASSWORD="12345"

LCD_ENDPOINT_MAIN = "https://fcd.terra.dev/v1/txs/"
CONTRACT = "terra1kc87mu460fwkqte29rquh4hc20m54fxwtsx7gp"

In [15]:
import psycopg2


def insert_txs(txs):
    connection = psycopg2.connect(user=DBUSER,
                                  password=DBPASSWORD,
                                  host=DBHOST,
                                  port=DBPORT)
    with connection:
        with connection.cursor() as cur:
            for tx in txs:
                cur.execute("insert into transactions(txhash,txdate,fcdtxid,txdata) values(%s,%s,%s,%s)",
                            (tx['txhash'], tx['timestamp'], tx['id'], json.dumps(tx)))


In [18]:
import requests
import datetime
import json


def fetch_txs(LCD_ENDPOIN,CONTRACT,offset=None):
    params = {
        "account": CONTRACT,
        "limit": 100
    }
    if offset is not None:
        params['offset']=offset
    i = 0
    until_day = datetime.date(2021, 5, 15)
    day = datetime.date.today()
    while day >= until_day:
        resp = requests.get(LCD_ENDPOIN, params=params)
        n = resp.json()['next']
        params['offset'] = n
        i += 1
        day = datetime.datetime.strptime(
            resp.json()['txs'][0]["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        print(resp.json()['txs'][0]["timestamp"])
        insert_txs(resp.json()['txs'])

connection = psycopg2.connect(user=DBUSER,
        password=DBPASSWORD,
        host=DBHOST,
        port=DBPORT)
cursor = connection.cursor()
cursor.execute("select min(fcdtxid) from transactions")
m = cursor.fetchone()
cursor.close()
connection.close()
txs = fetch_txs(LCD_ENDPOINT_MAIN,CONTRACT,m[0])

(129658865,)


In [ ]:
import copy
import json
import psycopg2


def save_msgs(msgs, message_type):
    connection = psycopg2.connect(user=DBUSER,
                                  password=DBPASSWORD,
                                  host=DBHOST,
                                  port=DBPORT)
    with connection:
        with connection.cursor() as cur:
            for msg in msgs:
                cur.execute("insert into messages(txhash,date,message_type,sender,recepient,body) values(%s,%s,%s,%s,%s,%s)",
                            (msg['hash'], msg['date'], message_type, msg['from'], msg['to'], json.dumps(msg)))


def parse_day(date):
    return datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").date()


def get_txs_from_db():
    connection = psycopg2.connect(user=DBUSER,
                                  password=DBPASSWORD,
                                  host=DBHOST,
                                  port=DBPORT)
    cursor = connection.cursor("qwerty")
    cursor.itersize = 1000
    cursor.execute("select txdata from transactions")
    rows = cursor.fetchmany(1000)
    c = 1
    while len(rows) > 0:
        print('processing ', c)
        c = c + 1
        for row in rows:
            yield row[0]
        rows = cursor.fetchmany(1000)
    cursor.close()
    connection.close()


def get_send_message_from_logs(tx, message):
    """ looking for "send" or "transfer" messages """
    msg = {
        'date': tx["timestamp"],
        'hash': tx['txhash'],
    }
    msgs = []
    type_detect_field = "action"
    if not 'logs' in tx:
        return []
    for log in tx['logs']:
        for e in log['events']:
            if e['type'] != "from_contract":
                continue
            match_contract = False
            match_message_type = False
            for a in e['attributes']:
                if a['key'] == "contract_address":
                    if match_contract and match_message_type:
                        msgs.append(copy.deepcopy(msg))
                    msg = {
                        'date': tx["timestamp"],
                        'hash': tx['txhash'],
                    }
                    match_contract = False
                    match_message_type = False
                    if a['value'] == CONTRACT:
                        match_contract = True
                    continue
                if match_contract and a['key'] == type_detect_field and a['value'] == message:
                    match_message_type = True
                    continue
                if match_contract and match_message_type:
                    msg[a['key']] = a['value']
            if match_contract and match_message_type:
                msgs.append(copy.deepcopy(msg))
    return msgs


message_type = "transfer"
messages = []
for tx in get_txs_from_db():
    msgs = get_send_message_from_logs(tx, message_type)
    messages = messages + msgs
save_msgs(messages, message_type)
print(len(messages))


In [7]:
import psycopg2
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as px
from plotly.subplots import make_subplots
from statistics import stdev, mean
import plotly.figure_factory as ff


def split_range(minValue, maxValue, parts):
    length = (maxValue - minValue)/parts
    splits = []
    for i in range(parts):
        splits.append((i*length, (i+1)*length))
    return splits


def distr_per_sender(days,min_txs):
    # latest transaction time in db - 2021-08-30 10:37:08+00
    # earliest trans - 2021-05-14 23:35:17+00
    date_clause = "and date > timestamp '2021-08-30 10:37:08+00' - interval '%s day'" % (days,)
    connection = psycopg2.connect(user=DBUSER,
                                  password=DBPASSWORD,
                                  host=DBHOST,
                                  port=DBPORT)
    cursor = connection.cursor("lalala")
    cursor.execute(
        """select 
            sender,
            count(sender) 
            from messages 
            where 
            message_type='send'
            %s
            group by sender 
            having count(sender)<100000 and count(sender)>%s""" % (date_clause,min_txs))
    senders = []
    trans_count = []
    for row in cursor.fetchall():
        senders.append(row[0])
        trans_count.append(row[1])

    ranges = split_range(min(trans_count)*0.9, max(trans_count)*1.1, 1000)
    # each dot actualy represents an interval ( s[0],s[1] ]
    x_axis = list(map(lambda s: (s[1]-s[0])/2+s[0], ranges))
    values = [0]*len(x_axis)
    s = stdev(trans_count)
    m = mean(trans_count)
    print(m, s)
    for value in trans_count:
        for r in enumerate(ranges):
            if value > r[1][0] and value <= r[1][1]:
                values[r[0]] = values[r[0]]+1
    figdev = go.Figure()
    figdev.add_trace(go.Scatter(x=x_axis, y=values, name="", mode="lines"))
    figdev.add_shape(type="rect",
                     x0=m, y0=0, x1=m+s, y1=max(values),
                     line=dict(
                         color="LightSalmon",
                         width=0,
                     ),
                     opacity=0.5,
                     fillcolor="LightSalmon",)
    figdev.add_shape(type="rect",
                     x0=m+s, y0=0, x1=m+s*2, y1=max(values),
                     line=dict(
                         color="LightSkyBlue",
                         width=0,
                     ),
                     opacity=0.5,
                     fillcolor="LightSkyBlue",)
    figdev.show()
    c = connection.cursor()
    for sigma_step in range(25):
        sigma = sigma_step/4
        c.execute(
            "select count(*) from (select sender,count(sender) from messages where message_type='send' %s group by sender having count(sender)>%s) as a" % (date_clause,m+s*sigma)
        )
        rec = c.fetchone()
        print("sigma = %f, accounts with amount of trans above %f  - %s" %
              (sigma, m+s*sigma, rec[0]))
    c.execute(
            "select sender,count(sender) from messages where message_type='send' %s group by sender order by count(sender) desc limit 5" % (date_clause,)
        )
    for row in c.fetchall():
        print(row)




def get_senders_amount(mix_txs):
    connection = psycopg2.connect(user=DBUSER,
                                  password=DBPASSWORD,
                                  host=DBHOST,
                                  port=DBPORT)
    cursor = connection.cursor()
    cursor.execute("select count(*) from (select sender from messages where message_type='send' group by sender having  count(*)>%s) as a",(mix_txs,))
    rec = cursor.fetchone()
    return int(rec[0])


def distr_per_day(sql):
    connection = psycopg2.connect(user=DBUSER,
                                  password=DBPASSWORD,
                                  host=DBHOST,
                                  port=DBPORT)
    cursor = connection.cursor("1")
    cursor.execute(sql)
    days = []
    sums = []
    amounts = []
    for row in cursor.fetchall():
        days.append(row[0])
        sums.append(row[1])
        amounts.append(row[2])

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=days, y=sums, name="Суммы по дням"),
                  secondary_y=False)
    fig.add_trace(go.Scatter(x=days, y=amounts,
                             name="кол-во"), secondary_y=True)
    fig.show()

    ranges = split_range(min(amounts)*0.9, max(amounts)*1.1, 30)
    # each dot actualy represents an interval ( s[0],s[1] ]
    x_axis = list(map(lambda s: (s[1]-s[0])/2+s[0], ranges))
    # x_axis = list(map(lambda s: "{:.2f}-{:.2f}".format(s[0],s[1]), ranges))
    values = [0]*len(x_axis)
    for value in amounts:
        for r in enumerate(ranges):
            if value > r[1][0] and value <= r[1][1]:
                values[r[0]] = values[r[0]]+1

    s = stdev(amounts)
    m = mean(amounts)
    print(m, s)

    figdev = go.Figure()
    figdev.add_trace(go.Scatter(x=x_axis, y=values, name="", mode="lines"))
    figdev.add_vrect(
        x0=m, x1=m+s,
        fillcolor="LightSalmon", opacity=0.5,
        layer="below", line_width=0,
    )
    figdev.add_vrect(
        x0=m+s, x1=m+s*2,
        fillcolor="LightGreen", opacity=0.5,
        layer="below", line_width=0,
    )

    figdev.show()


all_send = """select 
    date_trunc('day',date) as day ,
    sum((body->>'amount')::decimal),
    count(date) 
    from messages 
    where message_type='send' 
    group by day;"""
send_min100txs = """select 
    date_trunc('day',date) as day ,
    sum((body->>'amount')::decimal),
    count(date) 
    from messages 
    where message_type='send' 
    and sender in (select sender from messages where message_type='send' group by sender having count(*)>100)
    group by day;"""
# all_transfer="select date_trunc('day',date) as day ,sum((body->>'amount')::decimal),count(date) from messages where message_type='transfer' group by day;"
# distr_per_day(all_transfer)
distr_per_sender(150,8)
# distr_per_day(all_send)
distr_per_day(send_min100txs)
print("senders with at least 10 txs",get_senders_amount(10))

32.69347826086957 208.02495841244675


sigma = 0.000000, accounts with amount of trans above 32.693478  - 666
sigma = 0.250000, accounts with amount of trans above 84.699718  - 162
sigma = 0.500000, accounts with amount of trans above 136.705957  - 92
sigma = 0.750000, accounts with amount of trans above 188.712197  - 56
sigma = 1.000000, accounts with amount of trans above 240.718437  - 45
sigma = 1.250000, accounts with amount of trans above 292.724676  - 39
sigma = 1.500000, accounts with amount of trans above 344.730916  - 27
sigma = 1.750000, accounts with amount of trans above 396.737155  - 22
sigma = 2.000000, accounts with amount of trans above 448.743395  - 15
sigma = 2.250000, accounts with amount of trans above 500.749635  - 12
sigma = 2.500000, accounts with amount of trans above 552.755874  - 9
sigma = 2.750000, accounts with amount of trans above 604.762114  - 9
sigma = 3.000000, accounts with amount of trans above 656.768353  - 8
sigma = 3.250000, accounts with amount of trans above 708.774593  - 7
sigma = 3.

429.651376146789 851.790297355743


senders with at least 10 txs 2935
